**Demo for `teneva.core.act_one`**

---

This module contains the basic operations with one TT-tensor (Y), including "copy", "get", "sum", etc.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `copy`

Return a copy of the given TT-tensor.

In [2]:
Y = teneva.tensor_rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2

Z = teneva.copy(Y)         # The copy of Y

print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

-1.2208436499710222
-1.2208436499710222


In [3]:
Z[2][1, 2, 0] = 42.

print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

-1.2208436499710222
42.0


It also supports numbers for convenience:

In [4]:
teneva.copy(42.)

42.0

## Function `get`

Compute the element with multiindex $k$ from the TT-tensor $Y$. See also a function `tensor.getter` that performs the same operation, but with an acceleration.

In [5]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)           # Print the TT-tensor
k = [1, 2, 3, 4, 5]       # Select some tensor element
y1 = teneva.get(Y1, k)    # Compute the element of the TT-tensor
y0 = Y0[tuple(k)]         # Compute the same element of the original tensor
abs(np.max(y1-y0))        # Compare original tensor and reconstructed tensor

   10  10  10  10  10 
  / \ / \ / \ / \ / \ 
 1   10 100 100  10  1  



1.2212453270876722e-15

This function is also support batch mode:

In [6]:
K = [
    [1, 2, 3, 4, 5],
    [0, 0, 0, 0, 0],
    [5, 4, 3, 2, 1],
]

y1 = teneva.get(Y1, K)
y0 = [Y0[tuple(k)] for k in K]
abs(np.max(y1-y0))

9.992007221626409e-16

## Function `get_many`

Compute the elements of the TT-tensor on many indices (the function "get" with 2-dimensional argument may be used instead).

In [7]:
n = [10] * 5                    # Shape of the tensor      
Y0 = np.random.randn(*n)        # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)             # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)                 # Print the TT-tensor
K = [                           # Select some tensor elements
    [1, 2, 3, 4, 5],
    [0, 0, 0, 0, 0],
    [5, 4, 3, 2, 1],
]     
y1 = teneva.get_many(Y1, K)     # Compute the element of the TT-tensor
y0 = [Y0[tuple(k)] for k in K]  # Compute the same element of the original tensor
abs(np.max(y1-y0))              # Compare original tensor and reconstructed tensor

   10  10  10  10  10 
  / \ / \ / \ / \ / \ 
 1   10 100 100  10  1  



5.051514762044462e-15

## Function `getter`

Build fast (accelerated by numba) function that computes the element with multiindex $k$ for the TT-tensor $Y$. See also a function `tensor.get` for more details. Note that this function is not support the batch mode.

In [8]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
get = teneva.getter(Y1)   # Build (compile) function to compute the element of the TT-tensor
k = (1, 2, 3, 4, 5)       # Select some tensor element
y1 = get(k)               # Compute the element of the TT-tensor
y0 = Y0[k]                # Compute the same element of the original tensor
abs(np.max(y1-y0))        # Compare original tensor and reconstructed tensor

8.881784197001252e-16

We can compare the calculation time using the base function and the function accelerated with numba:

In [9]:
n = [100] * 40
Y = teneva.tensor_rand(n, r=4)

get1 = lambda i: teneva.get(Y, i)
get2 = teneva.getter(Y)

I = teneva.sample_lhs(n, m=1000)

t1 = tpc()
for i in I:
    y1 = get1(i)
t1 = tpc() - t1

t2 = tpc()
for i in I:
    y2 = get2(i)
t2 = tpc() - t2

print(f'Time for "simple" : {t1:-8.4f} sec')
print(f'Time for "numba"  : {t2:-8.4f} sec')

Time for "simple" :   0.1258 sec
Time for "numba"  :   0.0166 sec


## Function `mean`

Compute mean value of the TT-tensor.

In [10]:
Y = teneva.tensor_rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
m = teneva.mean(Y)                # The mean value

In [11]:
Y_full = teneva.full(Y)           # Compute tensor in the full format to check the result
m_full = np.mean(Y_full)          # The mean value for the numpy array
e = abs(m - m_full)               # Compute error for TT-tensor vs full tensor 
print(f'Error     : {e:-8.2e}')

Error     : 4.88e-19


The probability of tensor inputs my be also set:

In [12]:
n = [5]*10                        # Shape of the tensor
Y = teneva.tensor_rand(n, 2)      # 10-dim random TT-tensor with TT-rank 2
P = [np.zeros(k) for k in n]      # The "probability"
teneva.mean(Y, P)                 # The mean value

0.0

## Function `norm`

Compute Frobenius norm of the given TT-tensor.

In [13]:
Y = teneva.tensor_rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2

In [14]:
v = teneva.norm(Y)                # Compute the Frobenius norm
print(v)                          # Print the resulting value

27798.44414412251


In [15]:
Y_full = teneva.full(Y)           # Compute tensor in the full format to check the result

v_full = np.linalg.norm(Y_full)
print(v_full)                     # Print the resulting value from full tensor

e = abs((v - v_full)/v_full)      # Compute error for TT-tensor vs full tensor 
print(f'Error     : {e:-8.2e}')   # Rel. error

27798.444144122514
Error     : 1.31e-16


## Function `sum`

Compute sum of all tensor elements.

In [16]:
Y = teneva.tensor_rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.sum(Y)                                 # Sum of the TT-tensor elements

-1870.6371369853034

In [17]:
Z = teneva.full(Y) # Compute tensors in the full format to check the result
np.sum(Z)

-1870.6371369853034

---